In [44]:
import numpy

In [5]:
import pandas as pd

In [36]:
alldata=pd.read_csv('test.txt',sep='\t', header=1)
alldata

,#OTU ID,TMI3,PN1,PN3,PN2,pcrblank,500M2,500M3,negcontrol1,500M1,...,MRB1,TMI1,740RF3,740RF2,740RF1,TMI2,2KM3,1KM1,2KM1,taxonomy
0,557879642,0,487,0,0,1,0,0,0,1,...,3,0,0,0,1,0,0,0,3234,sc__Actinopterygii; c__Actinopteri; sc__Neopte...
1,557879642,0,487,0,0,1,0,0,0,1,...,3,0,0,0,1,0,0,0,3234,sc__Actinopterygii; c__Actinopteri; sc__Neopte...


In [37]:
labels=alldata[['#OTU ID','taxonomy']]
labels

,#OTU ID,taxonomy
0,557879642,sc__Actinopterygii; c__Actinopteri; sc__Neopte...
1,557879642,sc__Actinopterygii; c__Actinopteri; sc__Neopte...


In [38]:
controls=alldata[['pcrblank','negcontrol1']]
controls

,pcrblank,negcontrol1
0,1,0
1,1,0


In [39]:
data=alldata.drop(['#OTU ID','taxonomy','pcrblank','negcontrol1'],axis=1)
data

,TMI3,PN1,PN3,PN2,500M2,500M3,500M1,340RF3,340RF2,340RF1,...,MRB2,MRB1,TMI1,740RF3,740RF2,740RF1,TMI2,2KM3,1KM1,2KM1
0,0,487,0,0,0,0,1,0,0,0,...,1,3,0,0,0,1,0,0,0,3234
1,0,487,0,0,0,0,1,0,0,0,...,1,3,0,0,0,1,0,0,0,3234


In [56]:
def decontaminate_column(data_column,control_column):
    column = data_column.copy()
    column.ix[column <= control_column] = 0
    return column

In [57]:
decontaminate_column(data['500M1'], controls['pcrblank'])

0    0
1    0
Name: 500M1, dtype: int64

In [59]:
lambda c: decontaminate_column(c, controls['pcrblank'])

<function __main__.<lambda>>

In [65]:
data=data.apply(lambda c: decontaminate_column(c, controls['pcrblank']), axis=0)
data

,TMI3,PN1,PN3,PN2,500M2,500M3,500M1,340RF3,340RF2,340RF1,...,MRB2,MRB1,TMI1,740RF3,740RF2,740RF1,TMI2,2KM3,1KM1,2KM1
0,0,487,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3234
1,0,487,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3234


In [66]:
data=data.apply(lambda c: decontaminate_column(c, controls['negcontrol1']), axis=0)
data

,TMI3,PN1,PN3,PN2,500M2,500M3,500M1,340RF3,340RF2,340RF1,...,MRB2,MRB1,TMI1,740RF3,740RF2,740RF1,TMI2,2KM3,1KM1,2KM1
0,0,487,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3234
1,0,487,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3234


In [70]:
controls.columns.values.tolist()

['pcrblank', 'negcontrol1']

In [80]:
def decontaminate(dataframe,control):
    for control_name in control.columns.values.tolist():
        print(control_name)
        dataframe=dataframe.apply(lambda c: decontaminate_column(c, control[control_name]), axis=0)
    return dataframe

In [81]:
decontaminate(data,controls)

pcrblank
negcontrol1


,TMI3,PN1,PN3,PN2,500M2,500M3,500M1,340RF3,340RF2,340RF1,...,MRB2,MRB1,TMI1,740RF3,740RF2,740RF1,TMI2,2KM3,1KM1,2KM1
0,0,487,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3234
1,0,487,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,3234


In [83]:
alldata=pd.read_csv('otu_table_mc2_w_tax__Project_Moorea__renamed_text.txt',sep='\t', header=1)
labels=alldata[['#OTU ID','taxonomy']]
controls=alldata[['pcrblank','negcontrol1']]
data=alldata.drop(['#OTU ID','taxonomy','pcrblank','negcontrol1'],axis=1)
def decontaminate_column(data_column,control_column):
    column = data_column.copy()
    column.ix[column <= control_column] = 0
    return column

def decontaminate(dataframe,control):
    for control_name in control.columns.values.tolist():
        print(control_name)
        dataframe=dataframe.apply(lambda c: decontaminate_column(c, control[control_name]), axis=0)
    return dataframe
data=decontaminate(data,controls)

pcrblank
negcontrol1


In [96]:
cleaned_otu_table = pd.concat([labels['#OTU ID'],controls,data,labels['taxonomy']],axis=1)


In [107]:
with open('otu_table_mc2_w_tax__Project_Moorea__renamed_text.txt', 'r') as f:
    comment=f.readline()
    print(comment.strip())

# Constructed from biom file


In [108]:
with open('cleaned_otu_table.csv', 'w') as f:
    f.write(comment)

In [109]:
cleaned_otu_table.to_csv('cleaned_otu_table.csv', sep='\t', mode='a',index=False)